In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from PIL import Image
import ast
import os
import cv2
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_data = pd.read_csv("/kaggle/input/ranzcr-clip-catheter-line-classification/train.csv")
# CVC_train = train_data[["StudyInstanceUID","CVC - Abnormal", "CVC - Borderline", "CVC - Normal"]]
# CVC_train.set_index('StudyInstanceUID', inplace=True)
# CVC_train.head()

train_data[['CVC - Borderline', 'CVC - Normal', 'CVC - Abnormal']].value_counts()
train_data = train_data[train_data['CVC - Borderline'] != 1]
# train_data['CVC - Present'] = np.where((train_data['CVC - Abnormal'] == 0) & (train_data['CVC - Normal'] == 0) & (train_data['CVC - Borderline'] == 0), 0, 1)

# train_data['CVC - Present'].value_counts()
train_data['CVC - Borderline'].value_counts()
# train_data
# train_data['CVC - Borderline'] = np.where((train_data['CVC - Normal'] == 1) | (train_data['CVC - Abnormal'] == 1), 0, 0)

train_data['CVC - Present'] = np.where((train_data['CVC - Abnormal'] == 0) & (train_data['CVC - Normal'] == 0), 0, 1)

train_data['CVC - Present'].value_counts()

CVC - Present
1    20873
0      750
Name: count, dtype: int64

In [4]:
sampled_data = pd.concat([train_data[train_data['CVC - Present'] == 1].sample(n=750), train_data[train_data['CVC - Present'] == 0].sample(n=750)])
sampled_data

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,CVC - Present
1891,1.2.826.0.1.3680043.8.498.58681857182291021366...,0,0,0,0,0,0,1,0,0,1,0,3bd79fa0e,1
8892,1.2.826.0.1.3680043.8.498.26040311428295892195...,0,1,0,0,0,0,1,0,0,1,0,26da0d5ad,1
29814,1.2.826.0.1.3680043.8.498.10936927147214490186...,0,0,1,0,0,0,1,0,0,1,1,78c6b302b,1
21905,1.2.826.0.1.3680043.8.498.76559671062023300251...,0,0,0,0,0,0,0,0,0,1,0,5b17de379,1
11011,1.2.826.0.1.3680043.8.498.42883871312024967399...,0,0,0,0,0,0,0,0,0,1,0,0458f288e,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1,0
13342,1.2.826.0.1.3680043.8.498.12117973685813682040...,0,0,0,0,0,0,0,0,0,0,0,289b34706,0
30053,1.2.826.0.1.3680043.8.498.44243345248988583961...,0,0,1,0,0,1,0,0,0,0,0,6a490c713,0
10779,1.2.826.0.1.3680043.8.498.49214860405645081017...,0,0,0,0,1,0,0,0,0,0,0,f0384a6fa,0


In [5]:
def getFileNames(df):
    studyInstance = df['StudyInstanceUID'].tolist()
    studyInstance

    for i in range(len(studyInstance)):
        studyInstance[i] = f'{studyInstance[i]}.jpg'
      
    return studyInstance
    

In [6]:
studyInstance = getFileNames(sampled_data)

In [7]:
def processImages(filenames,train_test):

    imgs = []
    FileIDs = []
    lst = []


    desired_channels = 3

    count = 0
    for file in tqdm(filenames, position=0, leave=True):
        # Making sure the images are the same pixel size
        img = Image.open(f"/kaggle/input/ranzcr-clip-catheter-line-classification/{train_test}/{file}")
        img = img.resize((300, 300))
        img = np.array(img)

        # Ensure that the image has 3 channels (RGB)
        if img.shape[-1] != 3:
            img = np.stack((img,) * 3, axis=-1)  # Convert grayscale to RGB by duplicating channels

        imgs.append(img)  # Append the 4D image to the list

        # Save filename for later
        FileIDs.append(file)

        count += 1

    # Convert the list of images to a numpy array
    imgs = np.array(imgs)
    return imgs, FileIDs

In [8]:
train_imgs, fileIDs = processImages(studyInstance, 'train')

100%|██████████| 1500/1500 [01:13<00:00, 20.51it/s]


In [9]:
#cleaning up the labels
UID = []
for i in range(len(fileIDs)):
  UID.append(fileIDs[i].replace(".jpg",""))

UID[0:10]

['1.2.826.0.1.3680043.8.498.58681857182291021366107791259511816992',
 '1.2.826.0.1.3680043.8.498.26040311428295892195247941419133654159',
 '1.2.826.0.1.3680043.8.498.10936927147214490186520106312732173629',
 '1.2.826.0.1.3680043.8.498.76559671062023300251661629560732089635',
 '1.2.826.0.1.3680043.8.498.42883871312024967399805969755580957054',
 '1.2.826.0.1.3680043.8.498.17919616684931688691057562556057723286',
 '1.2.826.0.1.3680043.8.498.31325380162615235465376350291528393763',
 '1.2.826.0.1.3680043.8.498.10580478778264130081181863216602479420',
 '1.2.826.0.1.3680043.8.498.15133196733771703820438404110387411134',
 '1.2.826.0.1.3680043.8.498.14795931435159743627280419565930655107']

In [10]:
columns_to_drop = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Normal',
                  "NGT - Incompletely Imaged", "Swan Ganz Catheter Present", "PatientID", "CVC - Borderline", 'CVC - Abnormal', 'CVC - Normal']

CVCsampled_data = sampled_data.drop(columns=columns_to_drop, axis=1)

CVCsampled_data.set_index('StudyInstanceUID', inplace=True)

In [11]:
train_labels = np.empty((0,3))
data = []

for ID in UID:
    data.append(CVCsampled_data.loc[ID])

train_labels_df = pd.DataFrame(data)



train_labels_df = train_labels_df.reindex(UID)
train_labels = train_labels_df['CVC - Present'].values
# train_labels = train_labels_df.values
train_labels

array([1, 1, 1, ..., 0, 0, 0])

In [12]:
from sklearn.model_selection import train_test_split

# Split your data into training and temporary data (combined validation and test)
train_imgs, temp_imgs, train_labels, temp_labels = train_test_split(train_imgs, train_labels, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets
val_imgs, test_imgs, val_labels, test_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5, random_state=42)

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# CVC Detection

In [14]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Model


# from tensorflow.keras.layers import Dropout, BatchNormalization

# # Data Preparation
# # Load and preprocess your image data and labels here.

# # Load the ResNet50 model with pre-trained weights
# base_model = ResNet50(weights='imagenet', include_top=False)


# # Add dropout and batch normalization to custom fully connected layers
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# predictions = Dense(1, activation='sigmoid')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
# from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Reduce learning rate when the validation loss plateaus
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


# # Compile the model with optimizer and loss function
# model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

# # Early stopping to prevent overfitting
# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Save the best model during training
# model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# # Model Training with callbacks
# history = model.fit(train_imgs, train_labels, epochs=30, validation_data=(val_imgs, val_labels), callbacks=[reduce_lr, early_stop, model_checkpoint])

# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(test_imgs, test_labels)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [16]:
# # Make predictions
# predictions = model.predict(test_imgs)

# # Convert the predictions to a binary array
# predictions_binary = np.where(predictions > 0.5, 1, 0)

In [17]:
# from sklearn.metrics import confusion_matrix

# # Create the confusion matrix
# confusion_matrix = confusion_matrix(test_labels, predictions_binary)

In [18]:
# confusion_matrix


# plt.gca().patch.set_facecolor('none')



# custom_palette = ['#D3E0EA', '#AAB8C4', '#8498A2', '#5D7D88', '#3C6270', 
#                   '#1F465E', '#132D4B', '#0A1C36', '#051024', '#020916']

# # Create a heatmap plot of the confusion matrix
# sns.heatmap(confusion_matrix, annot=True, cmap=custom_palette)
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix')
# plt.show()


# plt.savefig('confusionmatrix_CVCexistence.png', bbox_inches='tight', transparent=True)

# NORMAL OR ABNORMAL

In [19]:
def buildTrainData(catheter, train_data, size):
    
    train_data = train_data[train_data[f'{catheter} - Present'] == 1]
    train_data = train_data = train_data[~((train_data[f'{catheter} - Normal'] == 1) & (train_data[f'{catheter} - Abnormal'] == 1))]
    train_data[f'{catheter} - Normal/Abnormal'] = np.where(train_data[f'{catheter} - Normal'] == 1, 1, 0)
    sampled_data = pd.concat([train_data[train_data[f'{catheter} - Normal'] == 1].sample(n=size), train_data[train_data[f'{catheter} - Abnormal'] == 1].sample(n=size)])
    
    return sampled_data
    

In [20]:
sample = buildTrainData('CVC', train_data, 750)
sample

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,CVC - Present,CVC - Normal/Abnormal
7563,1.2.826.0.1.3680043.8.498.46799292851414364343...,0,0,0,0,0,0,0,0,0,1,0,a3d1c2318,1,1
13,1.2.826.0.1.3680043.8.498.71008408169501434503...,0,0,0,0,0,0,0,0,0,1,0,7d2c4c9e3,1,1
21205,1.2.826.0.1.3680043.8.498.12303806657039898924...,0,1,0,0,0,1,0,0,0,1,0,a8b60f80b,1,1
23922,1.2.826.0.1.3680043.8.498.99215452685099644737...,0,0,0,0,0,0,0,0,0,1,0,0229b322d,1,1
16577,1.2.826.0.1.3680043.8.498.91313228181445130342...,0,0,0,0,0,0,0,0,0,1,0,77862c28a,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29091,1.2.826.0.1.3680043.8.498.61639060264216547988...,0,0,0,0,0,0,0,1,0,0,0,c2458a0b5,1,0
28593,1.2.826.0.1.3680043.8.498.68454648838987813579...,0,0,0,0,0,0,0,1,0,0,0,6d776746b,1,0
7167,1.2.826.0.1.3680043.8.498.13165420848972693483...,0,0,0,0,0,0,0,1,0,0,0,79baf9a2a,1,0
13219,1.2.826.0.1.3680043.8.498.35062349927682465189...,0,0,0,0,0,0,0,1,0,0,0,702f42b6a,1,0


In [21]:
nAb_data = getFileNames(sample)

cvc_imgs, fileIDs = processImages(nAb_data, 'train')

100%|██████████| 1500/1500 [01:11<00:00, 21.07it/s]


In [22]:
UID = []
for i in range(len(fileIDs)):
  UID.append(fileIDs[i].replace(".jpg",""))


columns_to_drop = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Normal',
                  "NGT - Incompletely Imaged", "Swan Ganz Catheter Present", "PatientID", "CVC - Borderline", 'CVC - Abnormal', 'CVC - Normal', 'CVC - Present']

CVCsample = sample.drop(columns=columns_to_drop, axis=1)

CVCsample.set_index('StudyInstanceUID', inplace=True)

CVCsample

,CVC - Normal/Abnormal
StudyInstanceUID,
1.2.826.0.1.3680043.8.498.46799292851414364343313356339958622616,1
1.2.826.0.1.3680043.8.498.71008408169501434503106701503871872312,1
1.2.826.0.1.3680043.8.498.12303806657039898924576833393381865693,1
1.2.826.0.1.3680043.8.498.99215452685099644737048838565704174635,1
1.2.826.0.1.3680043.8.498.91313228181445130342368263998568006846,1
...,...
1.2.826.0.1.3680043.8.498.61639060264216547988067122059977273656,0
1.2.826.0.1.3680043.8.498.68454648838987813579187819843696061681,0
1.2.826.0.1.3680043.8.498.13165420848972693483061373754459302220,0


In [23]:
train_labels_df = np.empty((0,3))
data = []

for ID in UID:
    data.append(CVCsample.loc[ID])

train_labels_df = pd.DataFrame(data)



train_labels_df = train_labels_df.reindex(UID)
cvc_labels = train_labels_df['CVC - Normal/Abnormal'].values
# train_labels = train_labels_df.values
cvc_labels

array([1, 1, 1, ..., 0, 0, 0])

In [24]:
from sklearn.model_selection import train_test_split

# Split your data into training and temporary data (combined validation and test)
trainCVC_imgs, temp_imgs, trainCVC_labels, temp_labels = train_test_split(cvc_imgs, cvc_labels, test_size=0.3, random_state=42)

# Split the temporary data into validation and test sets
valCVC_imgs, testCVC_imgs, valCVC_labels, testCVC_labels = train_test_split(temp_imgs, temp_labels, test_size=0.5, random_state=42)

In [25]:
# # Load the ResNet50 model with pre-trained weights
# base_model = ResNet50(weights='imagenet', include_top=False) ## If we get rid of this, does it add this model to the previous model? IDK 


# # Add dropout and batch normalization to custom fully connected layers
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# predictions = Dense(1, activation='sigmoid')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

# # Reduce learning rate when the validation loss plateaus
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


# # Compile the model with optimizer and loss function
# model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

# # Early stopping to prevent overfitting
# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Save the best model during training
# model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# # Model Training with callbacks
# history = model.fit(trainCVC_imgs, trainCVC_labels, epochs=30, validation_data=(val_imgs, val_labels), callbacks=[reduce_lr, early_stop, model_checkpoint])

# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(testCVC_imgs, testCVC_labels)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [26]:
# # Make predictions
# predictions = model.predict(test_imgs)

# # Convert the predictions to a binary array
# predictions_binary = np.where(predictions > 0.5, 1, 0)


# from sklearn.metrics import confusion_matrix

# # Create the confusion matrix
# confusion_matrix = confusion_matrix(test_labels, predictions_binary)

# plt.gca().patch.set_facecolor('none')



# custom_palette = ['#D3E0EA', '#AAB8C4', '#8498A2', '#5D7D88', '#3C6270', 
#                   '#1F465E', '#132D4B', '#0A1C36', '#051024', '#020916']

# # Create a heatmap plot of the confusion matrix
# sns.heatmap(confusion_matrix, annot=True, cmap=custom_palette)
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix')
# plt.show()


# plt.savefig('confusionmatrix_CVCexistence.png', bbox_inches='tight', transparent=True)



In [27]:
len(train_labels)

1050

In [28]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model


from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False)

# Add dropout and batch normalization to custom fully connected layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# First Output Branch: CVC Catheter Presence (Binary Classification)
output_catheter = Dense(1, activation='sigmoid', name='output_catheter')(x)

# Add a separate set of layers for the new task
y = base_model.output
y = GlobalAveragePooling2D()(y)
y = Dense(1024, activation='relu')(y)
y = BatchNormalization()(y)
y = Dropout(0.5)(y)

# Second Output Branch: CVC Placement (Binary Classification)
output_cvc_placement = Dense(1, activation='sigmoid', name='output_cvc_placement')(y)

# Combine the two output branches
combined_model = Model(inputs=base_model.input, outputs=[output_catheter, output_cvc_placement])

# Compile the combined model with optimizer and loss functions for both tasks
combined_model.compile(optimizer='SGD', 
                       loss={'output_catheter': 'binary_crossentropy', 'output_cvc_placement': 'binary_crossentropy'}, 
                       metrics={'output_catheter': 'accuracy', 'output_cvc_placement': 'accuracy'})

# # Reduce learning rate when the validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Save the best model during training
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

history = combined_model.fit(
    {'input_1': train_imgs},
    {'output_catheter': train_labels, 'output_cvc_placement': trainCVC_labels},
    epochs=30,
    validation_data=(
        {'input_1': val_imgs},
        {'output_catheter': val_labels, 'output_cvc_placement': valCVC_labels}
    ),
    callbacks=[reduce_lr, early_stop, model_checkpoint]
)

# # Evaluate the combined model
# test_loss, [test_accuracy_catheter, test_accuracy_cvc_placement] = combined_model.evaluate(
#     {'input_1': test_imgs},
#     {'output_catheter': test_labels, 'output_cvc_placement': testCVC_labels}
# )

# print(f'Test Accuracy (Catheter): {test_accuracy_catheter * 100:.2f}%')
# print(f'Test Accuracy (CVC Placement): {test_accuracy_cvc_placement * 100:.2f}%')

94765736/94765736 [==============================] - 4s 0us/step
Epoch 1/30
33/33 [==============================] - 41s 483ms/step - loss: 1.6623 - output_catheter_loss: 0.8400 - output_cvc_placement_loss: 0.8223 - output_catheter_accuracy: 0.6190 - output_cvc_placement_accuracy: 0.6429 - val_loss: 1.5317 - val_output_catheter_loss: 0.6828 - val_output_cvc_placement_loss: 0.8488 - val_output_catheter_accuracy: 0.5956 - val_output_cvc_placement_accuracy: 0.4844 - lr: 0.0100
Epoch 2/30
33/33 [==============================] - 12s 362ms/step - loss: 1.1047 - output_catheter_loss: 0.5328 - output_cvc_placement_loss: 0.5719 - output_catheter_accuracy: 0.7648 - output_cvc_placement_accuracy: 0.7505 - val_loss: 1.3521 - val_output_catheter_loss: 0.7247 - val_output_cvc_placement_loss: 0.6273 - val_output_catheter_accuracy: 0.5378 - val_output_cvc_placement_accuracy: 0.6400 - lr: 0.0100
Epoch 3/30
33/33 [==============================] - 12s 364ms/step - loss: 0.6164 - output_catheter_loss: 0

ValueError: too many values to unpack (expected 2)

In [35]:
# Evaluate accuracy for the 'output_catheter' branch
evaluation = combined_model.evaluate(
    {'input_1': test_imgs},
    {'output_catheter': test_labels, 'output_cvc_placement': testCVC_labels},
)

# Extract the accuracy values from the evaluation results
test_accuracy_catheter = test_loss_catheter[3]  # Index 3 corresponds to 'output_catheter_accuracy'
test_accuracy_cvc_placement = test_loss_catheter[4]  # Index 4 corresponds to 'output_cvc_placement_accuracy'

print(f'Test Accuracy (Catheter): {test_accuracy_catheter * 100:.2f}%')
print(f'Test Accuracy (CVC Placement): {test_accuracy_cvc_placement * 100:.2f}%')


8/8 [==============================] - 1s 80ms/step - loss: 1.4241 - output_catheter_loss: 0.6774 - output_cvc_placement_loss: 0.7466 - output_catheter_accuracy: 0.6489 - output_cvc_placement_accuracy: 0.6222
Test Accuracy (Catheter): 64.89%
Test Accuracy (CVC Placement): 62.22%
